let's load our data
PS: if the folowing line fails to execute you probably need to run "Feature engineering and exploration" notebook first

In [73]:
%store -r train_X
%store -r train_y
%store -r test
%store -r df_test
%store -r features_scores

In [74]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
%matplotlib inline

create a validation and train set

In [75]:
all_train_X, all_train_y = train_X, train_y
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, random_state=42, test_size=0.25)

It proves helpfull to feed our model only 150 of the top performing features

In [76]:
top_features = features_scores[:150]['feat'].values

train_X= (train_X[top_features])
val_X= (val_X[top_features])

test=(test[top_features])

In [68]:
regr = RandomForestRegressor(max_depth=100, random_state=42,criterion='mse', n_estimators=300)
regr.fit(train_X,train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [69]:
predictions = regr.predict(val_X)
predictions_train = regr.predict(train_X)
print("Root Mean Squarred Error Train: " + str(np.sqrt( mean_squared_error(predictions_train, train_y))))
print("Root Mean Squarred Error Valid: " + str(np.sqrt( mean_squared_error(predictions, val_y))))

Root Mean Squarred Error Train: 10914.326841873868
Root Mean Squarred Error Valid: 26478.87696748692


In [42]:
regr.score(train_X,train_y),regr.score(val_X,val_y)

(0.974470430694023, 0.894860457934442)

generate our submission file

In [77]:
regr = RandomForestRegressor(max_depth=100, random_state=42,criterion='mse', n_estimators=300)
regr.fit(all_train_X[top_features],all_train_y)

preds = regr.predict(test)
output=pd.DataFrame({'Id':df_test['Id'], 'SalePrice':preds})
output.to_csv('submission_forest.csv', index=False)

We do also export model's prediction to be used later with stacking and training a meta model

In [81]:
forest_predictions_train = regr.predict(all_train_X[top_features])
forest_predictions_test =regr.predict(test)
%store forest_predictions_train
%store forest_predictions_test

Stored 'forest_predictions_train' (ndarray)
Stored 'forest_predictions_test' (ndarray)
